# PlayHQ Fixture Scraping

This system allows to scrape games from [PlayHQ](http://playhq.com/
) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs) and produce CSV files ready to be uploaded to [TeamApp](https://brunswickmagicbasketball.teamapp.com/). (Private APIs are not available to clubs and associations.)

The *Public* APIs only require a header parameters to get a successful response, which includes `x-api-key` (also referred to as the Client ID) and `x-phq-tenant` (refers to the sport/association - in this case `bv`).

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

Contact Sebastian Sardina  (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re
import os
import calendar, datetime

import playhq_api as phq

# import dtale

## 1. Set-up application

We start by creating a connection to the PlayHQ Public API.

To do so, we need to define the ids and keys for the club to access the PlayHQ API.

The organization Play-HQ ID can be obtained from the admin PlayHQ URL:

```
https://bv.playhq.com/org/<ORG_ID>/competitions
```

The *Public* APIs only require the below header parameters to get a successful response:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.

The feature to create new API credentials is disabled for you and can only be actioned by a Super Administrator role within the Play HQ portal. Please use the credentials provided to call the PlayHQ public APIs.

In [2]:
CLUB_NAME = "Brunswick Magic Basketball Club"
X_TENANT="bv"
ORG_ID="8c4d5431-eaa5-4644-82ac-992abe224b88"
TIMEZONE = 'Australia/Melbourne'

GAME_DATE = phq.next_day(calendar.SATURDAY) # get the date of the upcoming Saturday (game day in competition)
GAME_DATE_TIMESTAMP = pd.to_datetime(GAME_DATE).tz_localize(TIMEZONE)
GAME_DATE_NAME = GAME_DATE_TIMESTAMP.strftime("%A %B %d, %Y (%Y/%m/%d)") # Saturday August 06, 2022



# Provide the x-api-key either explicitly or via a file
X_API_KEY="<YOUR PLAYHQ ADMIN KEY>"

if os.path.exists('x_api_key.txt'):
    with open('x_api_key.txt') as f:
        X_API_KEY = f.readlines()[0]

phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE)

print(f"Set-up games for club {CLUB_NAME} for upcoming Saturday is: {GAME_DATE_NAME}")

Set-up games for club Brunswick Magic Basketball Club for upcoming Saturday is: Saturday August 13, 2022 (2022/08/13)


Next define the data for the season that we are interested in.

In [3]:
SEASON = "Winter 2022"

DESC_BYE_TAPP = "Sorry, no game for the team in this round."
DESC_TAPP = """RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: {opponent}
Venue: {venue} ({court})
Address: {address} {address_tips}
Google Maps coord: https://maps.google.com/?q={coord}

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: {url_game}
Check the round in PlayHQ: {url_grade}
All clubs in PlayHQ: https://bit.ly/bmbc-w22
"""

PATH_SAVE_CSV='Brunswick_Magics/2022.01.Winter/fixture'

## 2. Get the teams and upcoming games for those teams

First, get the competition id for the season the organisation is in.

In [4]:
season_id = phq_club.get_season_id(SEASON)
teams_df = phq_club.get_season_teams(season_id)
teams_df

,id,name,grade.id,grade.name,grade.url,age
0,d7486008-b2fe-47db-8011-01265eaf1cfe,Magic U8 Mixed Purple,be7076cb-a1de-4033-8082-b52c8c149861,Saturday U8 Mixed Division 1/2,https://www.playhq.com/basketball-victoria/org...,8
1,8d635867-0718-4f70-a1b8-1a224992d294,Magic U16 Girls Gold,19b86372-2ad5-4a5a-9592-467f826f7d63,Saturday U16 Girls Division 1/2,https://www.playhq.com/basketball-victoria/org...,16
2,69d69214-641d-4ece-a2ce-ca9534865553,Magic U16 Boys Purple,be4b2b9c-0bc1-44e2-ac3a-b89bb10230f8,Saturday U16 Boys Division 3,https://www.playhq.com/basketball-victoria/org...,16
3,135c3bab-3004-4426-b9c6-3669b48ce271,Magic U16 Boys Gold,ceac1ec4-c86b-47c6-9e67-124d923ea713,Saturday U16 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,16
4,99f369ed-5c63-40e7-8591-8cd18fba7ec0,Magic U14 Boys Gold,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
5,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Magic U14 Girls Purple,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
6,9ebeb141-4023-4744-84f1-f58957b966b2,Magic U14 Girls Gold,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
7,2b844e90-6575-47bc-bb49-f55840f8e6f8,Magic U14 Girls Black,237a9d9a-52c5-4136-b179-c01b10ad93ae,Saturday U14 Girls Division 4,https://www.playhq.com/basketball-victoria/org...,14
8,623c00bb-302b-4492-a988-90aacc3e7768,Magic U14 Boys Purple,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
9,a1e571f6-8ac4-49d2-9787-f62b35947eca,Magic U14 Boys Black,b877a73b-037d-460b-acb5-a2c45e5debee,Saturday U14 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,14


Next get all upcoming games for the Club's teams.

In [5]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_TIMESTAMP)

print(f'There were {upcoming_games_df.size} games extracted for game day: {GAME_DATE_NAME}')
upcoming_games_df[phq.GAMES_COLS]

2022-08-10 18:19:22 INFO Games extracted for team: Magic U8 Mixed Purple
2022-08-10 18:19:23 INFO Games extracted for team: Magic U16 Girls Gold
2022-08-10 18:19:23 INFO Games extracted for team: Magic U16 Boys Purple
2022-08-10 18:19:23 INFO Games extracted for team: Magic U16 Boys Gold
2022-08-10 18:19:23 INFO Games extracted for team: Magic U14 Boys Gold
2022-08-10 18:19:24 INFO Games extracted for team: Magic U14 Girls Purple
2022-08-10 18:19:24 INFO Games extracted for team: Magic U14 Girls Gold
2022-08-10 18:19:24 INFO Games extracted for team: Magic U14 Girls Black
2022-08-10 18:19:25 INFO Games extracted for team: Magic U14 Boys Purple
2022-08-10 18:19:25 INFO Games extracted for team: Magic U14 Boys Black
2022-08-10 18:19:26 INFO Games extracted for team: Magic U12 Girls Gold
2022-08-10 18:19:27 INFO No games for team: Magic U12 Girls Black
2022-08-10 18:19:27 INFO Games extracted for team: Magic U12 Boys Purple
2022-08-10 18:19:27 INFO Games extracted for team: Magic U12 Boys

There were 576 games extracted for game day: Saturday August 13, 2022 (2022/08/13)


,team_name,status,schedule_timestamp,venue_name
0,U8 Mixed Purple,UPCOMING,2022-08-13 08:30:00+10:00,Oak Park Stadium
1,U16 Girls Gold,UPCOMING,2022-08-13 13:00:00+10:00,Coburg Basketball Stadium
2,U16 Boys Purple,UPCOMING,2022-08-13 13:45:00+10:00,Pascoe Vale Girls College
3,U16 Boys Gold,UPCOMING,2022-08-13 12:15:00+10:00,Mercy College
4,U14 Boys Gold,UPCOMING,2022-08-13 11:30:00+10:00,Northcote High School
5,U14 Girls Purple,UPCOMING,2022-08-13 11:30:00+10:00,Mercy College
6,U14 Girls Gold,UPCOMING,2022-08-13 13:00:00+10:00,Dallas Brooks Community Primary School
7,U14 Girls Black,UPCOMING,2022-08-13 13:00:00+10:00,St John's College (Preston)
8,U14 Boys Purple,UPCOMING,2022-08-13 11:30:00+10:00,Northcote High School
9,U14 Boys Black,UPCOMING,2022-08-13 10:45:00+10:00,Mercy College


## 3. Convert to TeamApp CSV format

Next, we convert tthe PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

In [12]:
games_tapps_df = phq.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP)
print("Done computing the games for Teams App")
games_tapps_df.sample(3)

Done computing the games for Teams App


,team_name,team_id,round_name,round_abbreviatedName,event_name,opponent,schedule_timestamp,start_date,end_date,start_time,...,attendance_tracking,duty_roster,ticketing,reference_id,venue,court,geo,game_url,grade_url,description
5,U14 Girls Purple,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Round 14,R14,U14 Girls Purple - Round 14,Jets U14 Girls Blue,2022-08-13 11:30:00+10:00,2022-08-13,2022-08-13,11:30:00,...,0,1,0,,Mercy College,Court 1,"(-37.72981,144.96403)",https://tinyurl.com/2ne6ncam,https://tinyurl.com/2cbtlglg,RSVP is YES by default - if you cannot make it...
16,U10 Girls Purple,f1b92c31-a2ef-4a31-953e-61f0a4e85f74,Round 10,R10,U10 Girls Purple - Round 10,SUBC-CG101,2022-08-13 09:15:00+10:00,2022-08-13,2022-08-13,09:15:00,...,0,1,0,,Oak Park Stadium,Court 1,"(-37.713213,144.925617)",https://tinyurl.com/2oxvjee8,https://tinyurl.com/2d5fput9,RSVP is YES by default - if you cannot make it...
11,U12 Boys Purple,2caba13f-f6bb-4919-a7e1-9561d9ce4354,Round 14,R14,U12 Boys Purple - Round 14,Piranhas U12 Boys Gold,2022-08-13 10:00:00+10:00,2022-08-13,2022-08-13,10:00:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 2,"(-37.67041,144.9418)",https://tinyurl.com/2on73on6,https://tinyurl.com/24tr9rwu,RSVP is YES by default - if you cannot make it...


Next filter and re-order columns relevant to Teams App schedule format.

In [7]:
TAPP_COLS_CSV = ['event_name', 'team_name', 'start_date', 'end_date', 'start_time', 'end_time', 'description', 'venue', 'location', 'access_groups', 'rsvp', 'comments', 'attendance_tracking', 'duty_roster', 'ticketing']

team_apps_csv_df = games_tapps_df.loc[:, TAPP_COLS_CSV]
team_apps_csv_df.sample(4)


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
3,U16 Boys Gold - Round 14,U16 Boys Gold,2022-08-13,2022-08-13,12:15:00,13:00:00,RSVP is YES by default - if you cannot make it...,Mercy College,"760 Sydney Road, Coburg",U16 Boys Gold,1,1,0,1,0
14,U12 Boys Black - Round 14,U12 Boys Black,2022-08-13,2022-08-13,10:45:00,11:30:00,RSVP is YES by default - if you cannot make it...,St John's College (Preston),"21 Railway Place West, Preston",U12 Boys Black,1,1,0,1,0
2,U16 Boys Purple - Round 14,U16 Boys Purple,2022-08-13,2022-08-13,13:45:00,14:30:00,RSVP is YES by default - if you cannot make it...,Pascoe Vale Girls College,"Cornwall Road, Pascoe Vale",U16 Boys Purple,1,1,0,1,0
1,U16 Girls Gold - Round 14,U16 Girls Gold,2022-08-13,2022-08-13,13:00:00,13:45:00,RSVP is YES by default - if you cannot make it...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U16 Girls Gold,1,1,0,1,0


In [8]:
# Inspect description of one record
print(team_apps_csv_df.iloc[4]['event_name'])
print(team_apps_csv_df.iloc[4]['description'])

U14 Boys Gold - Round 14
RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: Magic U14 Boys Purple
Venue: Northcote High School (Court 1)
Address: 19-25 St Georges Road, Northcote 
Google Maps coord: https://maps.google.com/?q=(-37.7742,144.98985)

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: https://tinyurl.com/2z9egfgm
Check the round in PlayHQ: https://tinyurl.com/24br3e8r
All clubs in PlayHQ: https://bit.ly/bmbc-w22



### Extract and add BYE games

In [9]:
# Extract the date of the round
date = team_apps_csv_df.iloc[1]['start_date']
date = GAME_DATE

print(f"Extract BYE games for games on {GAME_DATE_NAME}")

# Extract teams that do not have a game
playing_teams = games_tapps_df['team_id'].tolist()
bye_teams_df = teams_df.loc[~teams_df['id'].isin(playing_teams)].copy()

# Add entries for BYE teams
if not bye_teams_df.empty:  # there are BYE games
    print("There are BYE games...")

    bye_teams_df = bye_teams_df[['id', 'name']]

    bye_teams_df['team_name'] = bye_teams_df.apply(lambda x: re.search("U.*", x['name']).group(0), axis=1)
    bye_teams_df['access_groups'] = bye_teams_df['team_name']
    bye_teams_df['event_name'] = bye_teams_df['team_name'] + " - BYE"
    bye_teams_df['start_date'] = date
    bye_teams_df['end_date'] = date
    bye_teams_df['start_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['end_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['description'] = DESC_BYE_TAPP
    bye_teams_df['location'] = ""
    bye_teams_df['venue'] = "BYE"

    bye_teams_df['rsvp'] = 0
    bye_teams_df['comments'] = 0
    bye_teams_df['attendance_tracking'] = 0
    bye_teams_df['duty_roster'] = 0
    bye_teams_df['ticketing'] = 0
    bye_teams_df['reference_id'] = ""

    bye_teams_df = bye_teams_df[TAPP_COLS_CSV]
    team_apps_csv_df = pd.concat([team_apps_csv_df, bye_teams_df])
    team_apps_csv_df.drop_duplicates(inplace=True)


if bye_teams_df.size > 0:
    for x in bye_teams_df['team_name'].values:
        print(x)
else:
    print("No BYE games this round...")

bye_teams_df

Extract BYE games for games on Saturday August 13, 2022 (2022/08/13)
There are BYE games...
U12 Girls Black
U10 Girls Gold


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
11,U12 Girls Black - BYE,U12 Girls Black,2022-08-13,2022-08-13,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U12 Girls Black,0,0,0,0,0
18,U10 Girls Gold - BYE,U10 Girls Gold,2022-08-13,2022-08-13,00:00:00,00:00:00,"Sorry, no game for the team in this round.",BYE,,U10 Girls Gold,0,0,0,0,0


## 5. Save to CSV file for Teams App import

### 5.1. FINAL CHECK

Finally, report the games to be written into Schedule CSV file and **CHECK ALL IS GOOD!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [10]:
team_apps_csv_df.columns
team_apps_csv_df[['team_name', 'start_date', 'start_time', 'venue']]

,team_name,start_date,start_time,venue
0,U8 Mixed Purple,2022-08-13,08:30:00,Oak Park Stadium
1,U16 Girls Gold,2022-08-13,13:00:00,Coburg Basketball Stadium
2,U16 Boys Purple,2022-08-13,13:45:00,Pascoe Vale Girls College
3,U16 Boys Gold,2022-08-13,12:15:00,Mercy College
4,U14 Boys Gold,2022-08-13,11:30:00,Northcote High School
5,U14 Girls Purple,2022-08-13,11:30:00,Mercy College
6,U14 Girls Gold,2022-08-13,13:00:00,Dallas Brooks Community Primary School
7,U14 Girls Black,2022-08-13,13:00:00,St John's College (Preston)
8,U14 Boys Purple,2022-08-13,11:30:00,Northcote High School
9,U14 Boys Black,2022-08-13,10:45:00,Mercy College


### 5.2. Write a TeamAPP Schedule CSV

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [11]:
file_csv = os.path.join(PATH_SAVE_CSV, f"schedule-teamsapp-{date.strftime('%Y_%m_%d')}.csv")

print(f'Saving TeamAPP schedule CSV file for games on game date: {GAME_DATE_NAME}')
print('File to save TeamApp schedule:', file_csv)
team_apps_csv_df.to_csv(file_csv, index=False)

Saving TeamAPP schedule CSV file for games on game date: Saturday August 13, 2022 (2022/08/13)
File to save TeamApp schedule: Brunswick_Magics/2022.01.Winter/fixture/schedule-teamsapp-2022_08_13.csv


### 5.3. Write fixture dataframe too



In [17]:
import datetime
import os

now = datetime.datetime.now() # current date and time

now_str = now.strftime("%Y-%m-%d_%H:%M:%S")
upcoming_games_df.to_pickle(os.path.join(PATH_SAVE_CSV, f"upcoming_games_df-{now_str}.pkl"))
team_apps_csv_df.to_pickle(os.path.join(PATH_SAVE_CSV, f"team_apps_csv_df-{now_str}.pkl"))

print(f"Finished saving dataframes: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

Finished saving dataframes: 13/08/2022, 16:47:35


# ------------ END FIXTURE PUBLISHING ------------

## 6. Re-check Fixture

In [18]:
upcoming_games_df.query("team_name == 'U14 Girls Black'")

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp,venue
7,417b31de-9f91-4f4c-903c-dd14ce6a1dd8,U14 Girls Black,2b844e90-6575-47bc-bb49-f55840f8e6f8,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-06-12 22:19:23+10:00,2022-08-09 13:40:45+10:00,None,[{'id': '2b844e90-6575-47bc-bb49-f55840f8e6f8'...,237a9d9a-52c5-4136-b179-c01b10ad93ae,...,Crt1,21 Railway Place West,3072,Preston,VIC,Australia,-37.74845,144.99944,2022-08-13 13:00:00+10:00,NaN


In [19]:
upcoming_games2_df = phq_club.get_games(teams_df, GAME_DATE_TIMESTAMP)

print(f'There were {upcoming_games2_df.size} games extracted for game day: {GAME_DATE_NAME}')
upcoming_games_df[phq.GAMES_COLS]

2022-08-13 16:51:15 INFO Games extracted for team: Magic U8 Mixed Purple
2022-08-13 16:51:15 INFO Games extracted for team: Magic U16 Girls Gold
2022-08-13 16:51:16 INFO Games extracted for team: Magic U16 Boys Purple
2022-08-13 16:51:16 INFO Games extracted for team: Magic U16 Boys Gold
2022-08-13 16:51:16 INFO Games extracted for team: Magic U14 Boys Gold
2022-08-13 16:51:17 INFO Games extracted for team: Magic U14 Girls Purple
2022-08-13 16:51:17 INFO Games extracted for team: Magic U14 Girls Gold
2022-08-13 16:51:18 INFO Games extracted for team: Magic U14 Girls Black
2022-08-13 16:51:18 INFO Games extracted for team: Magic U14 Boys Purple
2022-08-13 16:51:19 INFO Games extracted for team: Magic U14 Boys Black
2022-08-13 16:51:19 INFO Games extracted for team: Magic U12 Girls Gold
2022-08-13 16:51:19 INFO No games for team: Magic U12 Girls Black
2022-08-13 16:51:20 INFO Games extracted for team: Magic U12 Boys Purple
2022-08-13 16:51:20 INFO Games extracted for team: Magic U12 Boys

There were 576 games extracted for game day: Saturday August 13, 2022 (2022/08/13)


,team_name,status,schedule_timestamp,venue_name
0,U8 Mixed Purple,UPCOMING,2022-08-13 08:30:00+10:00,Oak Park Stadium
1,U16 Girls Gold,UPCOMING,2022-08-13 13:00:00+10:00,Coburg Basketball Stadium
2,U16 Boys Purple,UPCOMING,2022-08-13 13:45:00+10:00,Pascoe Vale Girls College
3,U16 Boys Gold,UPCOMING,2022-08-13 12:15:00+10:00,Mercy College
4,U14 Boys Gold,UPCOMING,2022-08-13 11:30:00+10:00,Northcote High School
5,U14 Girls Purple,UPCOMING,2022-08-13 11:30:00+10:00,Mercy College
6,U14 Girls Gold,UPCOMING,2022-08-13 13:00:00+10:00,Dallas Brooks Community Primary School
7,U14 Girls Black,UPCOMING,2022-08-13 13:00:00+10:00,St John's College (Preston)
8,U14 Boys Purple,UPCOMING,2022-08-13 11:30:00+10:00,Northcote High School
9,U14 Boys Black,UPCOMING,2022-08-13 10:45:00+10:00,Mercy College


In [55]:
cols = ['team_name', 'schedule_timestamp', 'venue_name']

print("Report games that have changed since last extraction:")
pd.concat([upcoming_games2_df[cols], upcoming_games_df[cols]]).drop_duplicates(keep=False)

# upcoming_games2_df.columns




,team_name,schedule_timestamp,venue_name
7,U14 Girls Black,2022-08-13 13:00:00+10:00,Pascoe Vale Girls College
16,U10 Girls Purple,2022-08-13 00:00:00+10:00,NaN
7,U14 Girls Black,2022-08-13 13:00:00+10:00,St John's College (Preston)
16,U10 Girls Purple,2022-08-13 09:15:00+10:00,Oak Park Stadium
